# 2)	Talking to your data: Building an RAG system
Build a Retrieval-Augmented Generation (RAG) system to enhance our organization's data retrieval and content generation capabilities.

Importing necessary libraries

In [22]:
!pip install getpass4 pypdf llama-index openai faiss-cpu
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.0/887.0 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 31.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
  Created wheel for clipboard: filename=clipboard-0.0.4-py3-none-

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 5.8 MB/s eta 0:00:00


In [23]:
import os
from getpass import getpass

os.environ['OPENAI_API_KEY'] = getpass("OpenAI Key:")

OpenAI Key:··········


In [24]:
import nest_asyncio

nest_asyncio.apply()

In [25]:
from llama_index import SimpleDirectoryReader, ServiceContext, VectorStoreIndex, StorageContext
from llama_index.response.pprint_utils import pprint_response
from llama_index.llms import OpenAI
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine
import faiss
from llama_index.vector_stores import FaissVectorStore
from langchain.text_splitter import CharacterTextSplitter

## Load data
Downloaded from

https://www.homeworkforyou.com/static_media/uploadedfiles/Netflix%20Int%20Expansion.pdf

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
netflix = SimpleDirectoryReader(
    input_files=["/content/drive/MyDrive/Colab Notebooks/Netflix Int Expansion.pdf"]
).load_data()

## Configure base LLM service

Using Model **text-davinci-003**

In [28]:
#LLM instance - text-davinci-003 model

llm = OpenAI(temperature=0,
             model="text-davinci-003",
             max_tokens=500)

service_context = ServiceContext.from_defaults(llm=llm)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Build indices
Embeddings

Vector Store


In [29]:
#Dimensions of text-ada-embedding-002
d = 1536
faiss_index = faiss.IndexFlatL2(d)

In [30]:
#Vector Store
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
netflix_index = VectorStoreIndex.from_documents(netflix, storage_context=storage_context)

## Build query engines
For  text-davinci-003 model

In [31]:
netflix__engine = netflix_index.as_query_engine(similarity_top_k=2)

## Run queries

Checking for the document

In [32]:
response = netflix__engine.query(
    "What is the document about?"
)
print(response)

The document is about Netflix's international expansion and its challenges and successes in the global market.


Questions

1. What is Netflix give a brief answer?
2. What services does Netflix offer?
3. What is Netflix's flat fee?
4. Who is the Chief Executive Officer of Netflix? Give his brief introduction
5. What were some of the regulatory restrictions Netflix faced in expanding its services internationally in bullet points?
6. List all the challenges faced by Netflix out of America in different countries in bullet points?
7. How did Netflix's aggressive growth strategy come under criticism from industry analysts, and what were their concerns about the company's ability to provide area-specific, modified content to international subscribers?
8. In which year did Digital TV Viewers and Movie viewers gap was highest?
9. What were the major financial costs associated with Netflix's international expansion?
10. How much did Netflix offer to buy Blockbuster?

#Defining all the questions into variables

In [53]:
#Defining all the questions into variables

Quiz1 = "What is Netflix give a brief answer?"
Quiz2 = "What services does Netflix offer?"
Quiz3 = "What is Netflix's flat fee?"
Quiz4 = "Who is the Chief Executive Officer of Netflix? Give his brief introduction"
Quiz5 = "What were some of the regulatory restrictions Netflix faced in expanding its services internationally?"
Quiz6 = "List all the challenges faced by Netflix out of America in different countries?"
Quiz7 = "How did Netflix's aggressive growth strategy come under criticism from industry analysts, and what were their concerns about the company's ability to provide area-specific, modified content to international subscribers?"
Quiz8 = "In which year did Digital TV Viewers and Movie viewers gap was highest?"
Quiz9 = "What were the major financial costs associated with Netflix's international expansion?"
Quiz10 = "How much did Netflix offer to buy Blockbuster?"


## Answering questions using text-davinci-003 model

In [54]:
response1 = netflix__engine.query(Quiz1)
print(Quiz1,'\n', response1)

response2 = netflix__engine.query(Quiz2)
print(Quiz2,'\n', response2)

response3 = netflix__engine.query(Quiz3)
print(Quiz3,'\n', response3)

response4 = netflix__engine.query(Quiz4)
print(Quiz4,'\n', response4)

response5 = netflix__engine.query(Quiz5)
print(Quiz5,'\n', response5)

response6 = netflix__engine.query(Quiz6)
print(Quiz6,'\n', response6)

response7 = netflix__engine.query(Quiz7)
print(Quiz7,'\n', response7)

response8 = netflix__engine.query(Quiz8)
print(Quiz8,'\n', response8)

response9 = netflix__engine.query(Quiz9)
print(Quiz9,'\n', response9)

response10 = netflix__engine.query(Quiz10)
print(Quiz10,'\n', response10)


What is Netflix give a brief answer? 
 Netflix is a company that started as a DVD-mailing service and later transitioned to a subscription-based streaming service. It offers a digital library of movies, TV shows, and original content for a monthly fee. Netflix revolutionized the home entertainment industry by utilizing the internet and providing convenient access to a wide selection of content.
What services does Netflix offer? 
 Netflix offers a subscription-based service that provides access to a digital library of movies, television shows, and other original content. Additionally, Netflix used to offer a DVD-by-mail service, but it has shifted its focus to streaming services and video-on-demand (VOD).
What is Netflix's flat fee? 
 Netflix's flat fee is a monthly subscription fee that subscribers pay to have access to its digital library of movies, television shows, and other original content.
Who is the Chief Executive Officer of Netflix? Give his brief introduction 
 The Chief Exec

In [55]:
# Combine responses into a single string with new lines
all_responses = f"{response1}\n{response2}\n{response3}\n{response4}\n{response5}\n{response6}\n{response7}\n{response8}\n{response9}\n{response10}"

# Specify the file path where you want to save the text file
file_path = "/content/drive/MyDrive/Colab Notebooks/output_file.txt"

# Write the combined responses to the text file
with open(file_path, "w") as file:
    file.write(all_responses)

# Display a success message
print(f"All responses have been saved to: {file_path}")

All responses have been saved to: /content/drive/MyDrive/Colab Notebooks/output_file.txt


# Answering questions using gpt-4 model

In [56]:
#LLM instance - Model GPT-4
llm_gpt = OpenAI(temperature=0,
             model="gpt-4",
             max_tokens=500)

service_context = ServiceContext.from_defaults(llm=llm_gpt)

In [57]:
#Dimensions of text-ada-embedding-002
d = 1536
faiss_index = faiss.IndexFlatL2(d)

#Vector Store
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
netflix_index = VectorStoreIndex.from_documents(netflix, storage_context=storage_context)

#Query Engine
netflix__engine_gpt = netflix_index.as_query_engine(similarity_top_k=2)

In [58]:
#Sample question
response = netflix__engine_gpt.query(
    "What is the document about?"
)
print(response)

The document is about Netflix's international expansion and its challenges and successes in the global market.


In [59]:
#Querying on GPT engine

response1_gpt = netflix__engine_gpt.query(Quiz1)
print(Quiz1,'\n', response1_gpt)

response2_gpt = netflix__engine_gpt.query(Quiz2)
print(Quiz2,'\n', response2_gpt)

response3_gpt = netflix__engine_gpt.query(Quiz3)
print(Quiz3,'\n', response3_gpt)

response4_gpt = netflix__engine_gpt.query(Quiz4)
print(Quiz4,'\n', response4_gpt)

response5_gpt = netflix__engine_gpt.query(Quiz5)
print(Quiz5,'\n', response5_gpt)

response6_gpt = netflix__engine_gpt.query(Quiz6)
print(Quiz6,'\n', response6_gpt)

response7_gpt = netflix__engine_gpt.query(Quiz7)
print(Quiz7,'\n', response7_gpt)

response8_gpt = netflix__engine_gpt.query(Quiz8)
print(Quiz8,'\n', response8_gpt)

response9_gpt = netflix__engine_gpt.query(Quiz9)
print(Quiz9,'\n', response9_gpt)

response10_gpt = netflix__engine_gpt.query(Quiz10)
print(Quiz10,'\n', response10_gpt)



What is Netflix give a brief answer? 
 Netflix is a company that started as a DVD-mailing service and later transitioned to a subscription-based streaming service. It offers a digital library of movies, television shows, and original content for a flat monthly fee. Netflix has experienced significant growth and has become a major player in the home entertainment industry.
What services does Netflix offer? 
 Netflix offers a subscription-based service that provides access to a digital library of movies, television shows, and other original content. Additionally, Netflix used to offer a DVD-by-mail service, although it has shifted its focus towards internet streaming and video-on-demand (VOD) services.
What is Netflix's flat fee? 
 Netflix's flat fee is a monthly subscription fee that subscribers pay to have access to its digital library of movies, television shows, and other original content.
Who is the Chief Executive Officer of Netflix? Give his brief introduction 
 The Chief Executiv

In [60]:
# Combine responses into a single string with new lines
all_responses_gpt = f"{response1_gpt} \n{response2_gpt} \n{response3_gpt} \n{response4_gpt} \n{response5_gpt} \n{response6_gpt} \n{response7_gpt} \n{response8_gpt} \n{response9_gpt} \n{response10_gpt}"

# Write the combined responses to the text file
with open("/content/drive/MyDrive/Colab Notebooks/output_file_gpt.txt", "w") as file:
    file.write(all_responses_gpt)

# Display a success message after writing the file
print(f"All responses have been saved to: {file_path}")

All responses have been saved to: /content/drive/MyDrive/Colab Notebooks/output_file.txt


# BERTScore

# For calculation BERT Score
Using model text-davinci-003 model vs gpt-4

In [61]:
#Installing necessary library for calculating BERTScore
!pip install torchmetrics
!pip install transformers
from pprint import pprint
from torchmetrics.text.bert import BERTScore
from torchmetrics.functional.text.bert import bert_score

In [62]:
#Reading responses to find BERT Score of the responses

#text-davinci-003 model
with open("/content/drive/MyDrive/Colab Notebooks/output_file.txt") as f:
    cands = [line.strip() for line in f]

#GPT-4 Model
with open("/content/drive/MyDrive/Colab Notebooks/output_file_gpt.txt") as f:
    refs = [line.strip() for line in f]

In [43]:
#checking file from text-davinci-003 model
cands[0]

'Netflix is a company that started as a DVD-mailing service and later transitioned to a subscription-based streaming service. It offers a digital library of movies, television shows, and original content for a flat monthly fee. Netflix has experienced significant growth and has become a major player in the home entertainment industry.'

In [44]:
#checking file from GPT-4 Model
refs[0]

'Netflix is a company that started as a DVD-mailing service and later transitioned to a subscription-based streaming service. It offers a digital library of movies, television shows, and original content for a flat monthly fee. Netflix has experienced significant growth and has become a major player in the home entertainment industry.'

In [63]:
#BERT Score obtained from both the models text-davinci-003 model vs gpt-4
bertscore = BERTScore()
pprint(bertscore(cands, refs))

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: The argument `model_name_or_path` was not specified while it is required when the default `transformers` model is used. It will use the default recommended model - 'roberta-large'.
  warnings.warn(*args, **kwargs)  # noqa: B028
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'f1': tensor([0.9841, 0.9536, 1.0000, 1.0000, 0.9867, 1.0000, 0.9562, 1.0000, 0.9552,
        0.9560]),
 'precision': tensor([0.9833, 0.9574, 1.0000, 1.0000, 0.9866, 1.0000, 0.9537, 1.0000, 0.9558,
        0.9533]),
 'recall': tensor([0.9849, 0.9498, 1.0000, 1.0000, 0.9869, 1.0000, 0.9586, 1.0000, 0.9545,
        0.9587])}


# BertScore for Baseline Reponses with text-davinci-003 model vs baseline responses


## Baseline Responses

#### Answer 1
Netflix is a publicly traded company that offers subscription video streaming and online digital video disc (DVD) and Blu-ray Disc rental services.

#### Answer 2
Netflix primarily offers a subscription-based streaming service that allows users to watch movies, TV shows, and other content on demand over the internet.
#### Answer 3
The company's digital library includes a wide range of content, including original programming produced by Netflix, as well as licensed content from other studios and networks. In addition to its streaming service, Netflix also offers a DVD-by-mail rental service in the United States, although this service has become less important to the company in recent years.

#### Answer 4
Netflix's flat monthly subscription fee varies depending on the country and the plan chosen by the subscriber.
Chief Executive Officer (CEO) of Netflix is Reed Hastings. Hastings co-founded Netflix in 1997 and has been the CEO since 1998. He is a well-known entrepreneur and philanthropist who has received numerous awards for his contributions to the technology and entertainment industries. Hastings has been instrumental in shaping Netflix's business model and leading the company's international expansion efforts. Under his leadership, Netflix has become a dominant player in the streaming services industry, with millions of subscribers worldwide.

#### Answer 5
Netflix faced various regulatory restrictions in different countries that it chose for expansion. In Indonesia, the country's censorship agency claimed that much of Netflix's content was unsuitable for local audiences. Vietnamese regulators were also cautious on similar grounds, and Malaysia was likely to require that Netflix follow its censorship policy, largely due to religious reasons. China, in spite of its highly attractive market with its enormous, broadband-capable population, was very difficult to operate in because of regulatory challenges. More specifically, entering the Chinese market had proven challenging for Netflix due to strict regulations imposed on the country's media and entertainment industries.

#### Answer 6
The challenges faced by Netflix in expanding its services internationally include:1. Regulatory restrictions: Netflix was subject to both U.S. and host country regulations, and it needed to adapt its content offerings to serve the local customers' needs. The impact of these restrictions was apparent in its January 2016 expansion announcement, with countries like Syria absent from its line-up.Local competition: In spite of its high brand recognition in the United States, Netflix faced severe competition from local pay television operators and VOD service providers in many countries. 3.Local adaptation: The adaptation of content to suit regional markets was another critical challenge in Netflix's international expansion. Netflix's aggressive growth strategy came under criticism from industry analysts. They claimed that Netflix was outpacing its ability to provide area-specific, modified content to international subscribers and to develop market penetration strategies that were specific to the host country. 4. Funding: Due to these factors, Netflix was expecting to have to make a substantial investment to implement its ambitious international expansion plan. 5. Regulatory restrictions in specific countries: For example, in Indonesia, the country's censorship agency claimed that much of Netflix's content was unsuitable for local audiences. Vietnamese regulators were also cautious on similar grounds, and Malaysia was likely to require that Netflix follow its censorship policy, largely due to religious reasons. China, in spite of its highly attractive market with its enormous, broadband-capable population, was very difficult to operate in because of regulatory challenges. More specifically, entering the Chinese market had proven challenging for Netflix due to strict regulations imposed on the country's media and entertainment industries.

#### Answer 7
Industry analysts criticized Netflix's aggressive growth strategy, claiming that the company was outpacing its ability to provide area-specific, modified content to international subscribers and to develop market penetration strategies that were specific to the host country. The analysts were concerned that Netflix's one-size-fits-all approach to content offerings would not work in different countries, where audiences had strong preferences for local-language and local-content service offerings. They argued that Netflix needed to invest more in creating region-specific content and developing market penetration strategies that were tailored to the local market environment. The analysts also pointed out that Netflix faced severe competition from local pay television operators and VOD service providers in many countries, which made it difficult for the company to gain a foothold in these markets.

#### Answer 8
The gap between Digital TV Viewers and Movie viewers was highest in the year 2013.

#### Answer 9
The major financial costs associated with Netflix's international expansion included direct costs incurred by the company, fees for global licensing deals, the cost of broadband and Internet availability, and the ability of the country's infrastructure to support widespread streaming services. To address the challenges of licensing, distribution, and technology costs in different markets, Netflix adopted a multi-pronged approach. The company partnered with local content providers, invested heavily in marketing and distribution, developed region-specific content, and worked closely with local governments and regulatory bodies to ensure compliance with local laws and regulations.

#### Answer 10
Netflix offered to sell itself to Blockbuster for $50 million in the year 2000

In [46]:
#Writing reponses in text file
Ans1 = "Netflix is a publicly traded company that offers subscription video streaming and online digital video disc (DVD) and Blu-ray Disc rental services."
Ans2 = "Netflix primarily offers a subscription-based streaming service that allows users to watch movies, TV shows, and other content on demand over the internet. The company's digital library includes a wide range of content, including original programming produced by Netflix, as well as licensed content from other studios and networks. In addition to its streaming service, Netflix also offers a DVD-by-mail rental service in the United States, although this service has become less important to the company in recent years."
Ans3 = "Netflix's flat monthly subscription fee varies depending on the country and the plan chosen by the subscriber. "
Ans4 = "Chief Executive Officer (CEO) of Netflix is Reed Hastings. Hastings co-founded Netflix in 1997 and has been the CEO since 1998. He is a well-known entrepreneur and philanthropist who has received numerous awards for his contributions to the technology and entertainment industries. Hastings has been instrumental in shaping Netflix's business model and leading the company's international expansion efforts. Under his leadership, Netflix has become a dominant player in the streaming services industry, with millions of subscribers worldwide."
Ans5= "Netflix faced various regulatory restrictions in different countries that it chose for expansion. In Indonesia, the country's censorship agency claimed that much of Netflix's content was unsuitable for local audiences. Vietnamese regulators were also cautious on similar grounds, and Malaysia was likely to require that Netflix follow its censorship policy, largely due to religious reasons. China, in spite of its highly attractive market with its enormous, broadband-capable population, was very difficult to operate in because of regulatory challenges. More specifically, entering the Chinese market had proven challenging for Netflix due to strict regulations imposed on the country's media and entertainment industries."
Ans6 = "The challenges faced by Netflix in expanding its services internationally include:1. Regulatory restrictions: Netflix was subject to both U.S. and host country regulations, and it needed to adapt its content offerings to serve the local customers' needs. The impact of these restrictions was apparent in its January 2016 expansion announcement, with countries like Syria absent from its line-up.Local competition: In spite of its high brand recognition in the United States, Netflix faced severe competition from local pay television operators and VOD service providers in many countries. 3.Local adaptation: The adaptation of content to suit regional markets was another critical challenge in Netflix's international expansion. Netflix's aggressive growth strategy came under criticism from industry analysts. They claimed that Netflix was outpacing its ability to provide area-specific, modified content to international subscribers and to develop market penetration strategies that were specific to the host country. 4. Funding: Due to these factors, Netflix was expecting to have to make a substantial investment to implement its ambitious international expansion plan. 5. Regulatory restrictions in specific countries: For example, in Indonesia, the country's censorship agency claimed that much of Netflix's content was unsuitable for local audiences. Vietnamese regulators were also cautious on similar grounds, and Malaysia was likely to require that Netflix follow its censorship policy, largely due to religious reasons. China, in spite of its highly attractive market with its enormous, broadband-capable population, was very difficult to operate in because of regulatory challenges. More specifically, entering the Chinese market had proven challenging for Netflix due to strict regulations imposed on the country's media and entertainment industries."
Ans7 = "Industry analysts criticized Netflix's aggressive growth strategy, claiming that the company was outpacing its ability to provide area-specific, modified content to international subscribers and to develop market penetration strategies that were specific to the host country. The analysts were concerned that Netflix's one-size-fits-all approach to content offerings would not work in different countries, where audiences had strong preferences for local-language and local-content service offerings. They argued that Netflix needed to invest more in creating region-specific content and developing market penetration strategies that were tailored to the local market environment. The analysts also pointed out that Netflix faced severe competition from local pay television operators and VOD service providers in many countries, which made it difficult for the company to gain a foothold in these markets."
Ans8 = "The gap between Digital TV Viewers and Movie viewers was highest in the year 2013."
Ans9 = "The major financial costs associated with Netflix's international expansion included direct costs incurred by the company, fees for global licensing deals, the cost of broadband and Internet availability, and the ability of the country's infrastructure to support widespread streaming services. To address the challenges of licensing, distribution, and technology costs in different markets, Netflix adopted a multi-pronged approach. The company partnered with local content providers, invested heavily in marketing and distribution, developed region-specific content, and worked closely with local governments and regulatory bodies to ensure compliance with local laws and regulations."
Ans10 = "Netflix offered to sell itself to Blockbuster for $50 million in the year 2000"

In [47]:
# Combine responses into a single string with newline
answers = f"{Ans1} \n{Ans2} \n{Ans3} \n{Ans4} \n{Ans5} \n{Ans6} \n{Ans7} \n{Ans8} \n{Ans9} \n{Ans10}"
# Specify the file path where you want to save the text file
file_path = "/content/drive/MyDrive/Colab Notebooks/refs.txt"
# Write the combined responses to the text file
with open("/content/drive/MyDrive/Colab Notebooks/refs.txt", "w") as file:
    file.write(answers)

# Display a success message
print(f"All responses have been saved to: {file_path}")

All responses have been saved to: /content/drive/MyDrive/Colab Notebooks/refs.txt


In [64]:
#Reading responses to find BERT Score of the responses

#text-davinci-003 model
with open("/content/drive/MyDrive/Colab Notebooks/output_file.txt") as f:
    cands = [line.strip() for line in f]

#Baseline responses
with open("/content/drive/MyDrive/Colab Notebooks/refs.txt") as f:
    refs = [line.strip() for line in f]

In [49]:
#checking file from text-davinci-003 model
cands[0]

'Netflix is a company that started as a DVD-mailing service and later transitioned to a subscription-based streaming service. It offers a digital library of movies, television shows, and original content for a flat monthly fee. Netflix has experienced significant growth and has become a major player in the home entertainment industry.'

In [50]:
#checking file from baseline responses
refs[0]

'Netflix is a publicly traded company that offers subscription video streaming and online digital video disc (DVD) and Blu-ray Disc rental services.'

In [65]:
#BERT Score obtained from both the models text-davinci-003 model vs baseline answers
#bertscore = BERTScore()
pprint(bertscore(cands,refs))

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'f1': tensor([0.9614, 0.9592, 0.9658, 0.9985, 0.9681, 0.9913, 0.9652, 0.9598, 0.9522,
        0.9517]),
 'precision': tensor([0.9650, 0.9604, 0.9635, 0.9985, 0.9716, 0.9941, 0.9592, 0.9642, 0.9563,
        0.9521]),
 'recall': tensor([0.9578, 0.9580, 0.9680, 0.9985, 0.9646, 0.9885, 0.9713, 0.9554, 0.9482,
        0.9513])}
